In [5]:
import cv2
import numpy as np
from pathlib import Path

# Code for Enter key: 13
# Code for Nothing: 255
# Code for Escape: 27

import winsound
frequency = 300  # Set Frequency To 2500 Hertz
duration = 1000  # Set Duration To 1000 ms == 1 second

background = None
alpha = 0.5 
'''A lower value for this variable means 
running average will be performed over a larger amount of 
previous frames and vice-versa.'''

ROI_top = 100
ROI_bottom = 300
ROI_right = 150
ROI_left = 350
ROI = {'top':100,'bottom':300,'right':150,'left':350}

In [6]:
def calc_background(frame, alpha):

    global background
    
    if background is None:
        background = frame.copy().astype("float")
        return None
    
    '''The objective is to detect active objects from the difference obtained from 
    the reference frame and the current frame (background). We keep feeding each frame to the accumulateWeighted() function,
    and the function keeps finding the averages of all frames. '''
    
    cv2.accumulateWeighted(frame, background, alpha)

In [7]:
def detect_hand(frame, threshold=25):
    global background
    
    diff = cv2.absdiff(background.astype("uint8"), frame) #absolute difference between hand frame and background frame

    _ , thresholded = cv2.threshold(diff, threshold, 255, cv2.THRESH_BINARY) #if pixel value >=255, 1 else 0 (Binary thresholding)

    # detects change in the image color and marks it as contour. Detection of hand before the background
    contours, hierarchy = cv2.findContours(thresholded.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE) 
    # hierarchy: contains information about image topology; number of elements equal to number of contours

    if len(contours) == 0:
        return None
    else:
        
        hand_segment_max_cont = max(contours, key=cv2.contourArea) #sorts by contour area and finds contour with maximum area
        
        return (thresholded, hand_segment_max_cont)

In [12]:
f = True
element = 0
cam = cv2.VideoCapture(0)
while f:
    
    ret, frame = cam.read()

    # flipping the frame to prevent inverted image of captured frame...
    frame = cv2.flip(frame, 1)

    frame_copy = frame.copy()
    cv2.putText(frame_copy, "Enter next choice of character", (0, 400), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255), 2)
    cv2.putText(frame_copy, "Press Escape to exit", (0, 450), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255), 2)
    cv2.imshow("Sign Language Image Data Collector", frame_copy)
    num_frames = 0
    ele0 = cv2.waitKey(1) & 0xFF
    if ele0==27: #break loop for escape
        print("breaking loop")
        break
    element = chr(ele0)
    #element = 'A'
    num_imgs_taken = 0
    print(chr(ele0),ele0) if ele0!=255 else print(end="")
    while ele0!=255 and ele0!=13:
        ret, frame = cam.read()

        # flipping the frame to prevent inverted image of captured frame...
        frame = cv2.flip(frame, 1)

        frame_copy = frame.copy()

        roi = frame[ROI['top']:ROI_bottom, ROI_right:ROI_left]

        gray_frame = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
        gray_frame = cv2.GaussianBlur(gray_frame, (9, 9), 0)

        if num_frames < 60:
            calc_background(gray_frame, alpha)
            cv2.putText(frame_copy, "Reading background data...", (80, 400), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0,0,255), 2)
            
        #Time to configure the hand specifically into the ROI...
        elif num_frames <= 300: 

            hand = detect_hand(gray_frame)
            
            cv2.putText(frame_copy, "Make gesture for '"+str(element)+"'", (200, 400), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255), 2)
            
            # Checking if hand is actually detected by counting number of contours detected...
            if hand is not None:
                
                thresholded, hand_segment = hand

                # Draw contours around hand segment
                cv2.drawContours(frame_copy, [hand_segment + (ROI_right, ROI['top'])], -1, (255, 0, 0),1)
                
                cv2.putText(frame_copy, str(num_frames)+" for " + str(element), (70, 45), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255), 2)

                # Also display the thresholded image
                cv2.imshow("Thresholded Hand Image", thresholded)
        
        else: 
            
            # Segmenting the hand region...
            hand = detect_hand(gray_frame)
            # Checking if we are able to detect the hand...
            if hand is not None:
                
                # unpack the thresholded img and the max_contour...
                thresholded, hand_segment = hand

                # Drawing contours around hand segment
                cv2.drawContours(frame_copy, [hand_segment + (ROI_right, ROI['top'])], -1, (255, 0, 0),1)
                
                
                cv2.putText(frame_copy, str(num_imgs_taken) + 'images for' + str(element), (200, 400), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255), 2)
                
                # Displaying the thresholded image
                cv2.imshow("Thresholded Hand Image", thresholded)
                if num_imgs_taken <= 500:
                    cv2.putText(frame_copy, 'Collected '+str(num_imgs_taken)+' training images for' + str(element), (0, 450), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255), 2)
#                     Path("gesture/train/"+str(element)+"/").mkdir(parents=True, exist_ok=True)
#                     printed = cv2.imwrite(r"data2/train/"+str(element)+"/"+str(num_imgs_taken) + '.jpg', thresholded)
#                     print(printed," img: ",num_imgs_taken ) if not printed else print(end="")
#                     pass
                elif num_imgs_taken<=600:
                    cv2.putText(frame_copy, 'Collected '+str(num_imgs_taken)+' testing images for' + str(element), (0, 450), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255), 2)
#                     Path("gesture/test/"+str(element)+"/").mkdir(parents=True, exist_ok=True)
#                     printed = cv2.imwrite(r"data2/test/"+str(element)+"/"+str(num_imgs_taken) + '.jpg', thresholded)
#                     print(printed," img: ",num_imgs_taken ) if not printed else print(end="")
#                     pass
                else:
                    break
                num_imgs_taken+=1
            else:
                cv2.putText(frame_copy, 'No hand detected...', (200, 400), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255), 2)

        # Drawing ROI on frame copy
        cv2.rectangle(frame_copy, (ROI_left, ROI['top']), (ROI_right, ROI_bottom), (255,128,0), 3)
        
        cv2.putText(frame_copy, "CSE4020 J Component - Sign Language Detection & Recognition", (10, 20), cv2.FONT_ITALIC, 0.5, (51,255,51), 1)
        
        # increment the number of frames for tracking
        num_frames += 1

        # Display the frame with segmented hand
        cv2.imshow("Sign Language Image Data Collector", frame_copy)

        # Closing windows with Esc key...(any other key with ord can be used too.)
        k = cv2.waitKey(1) & 0xFF
        if k == 27:
            break
    

    #f = False
# Releasing camera & destroying all the windows...
cv2.destroyAllWindows()
cam.release()

1 49
breaking loop


In [14]:
cv2.destroyAllWindows()
cam.release()